In [1]:
import os
import sys

import tensorflow as tf
from keras.models import load_model
from pathlib import Path
import pandas as pd
import numpy as np

# import data_generator from parent directory
sys.path.append("..")
from training_pipeline.data_generator import preprocess_image
from training_pipeline.data_generator import CropType


MODEL_PATH = os.path.join(os.getcwd(), "../cache/keras_model/saved_model_rgb_classifier")
TEST_FILES_METADATA_CSV = "/Volumes/FD/ai_or_not/data/cifake/test/FAKE_metadata.csv"

REPO_ID = "ai-or-not/ai-or-not-model"

In [2]:
# load tensorflow saved model
# model = tf.keras.models.load_model(MODEL_PATH)
import tensorflow as tf
import tensorflow_hub as hub

# model = tf.saved_model.load(MODEL_PATH)
model = tf.keras.models.load_model(MODEL_PATH)
# model =tf.keras.layers.TFSMLayer(MODEL_PATH, call_endpoint="serving_default")

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  METAL, no compute capability (probably not an Nvidia GPU)
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2025-01-07 12:56:20.523890: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2025-01-07 12:56:20.523924: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-01-07 12:56:20.523932: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2025-01-07 12:56:20.523961: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-07 12:56:20.523978: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-01-07 12:56:20 WARNING  Mixed precision compatibility check (mixed_float16): WARNING
Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute ca

In [3]:
model.summary()

Model: "rgb_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 efficientnetv2-s (Function  (None, 7, 7, 1280)        20331360  
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 top_dropout (Dropout)       (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 3)                 3843      
                                                                 
 predictions (Activation)    (None, 3)              

In [4]:
metadata = pd.read_csv(TEST_FILES_METADATA_CSV, skiprows=1)
test_files = metadata.values
test_files = test_files[:10]

# extract prefix path from TEST_FILES_METADATA_CSV
prefx_path = Path(TEST_FILES_METADATA_CSV).parent
for file in test_files:
    file = file[0]
    image = preprocess_image(
        os.path.join(prefx_path, file), 
        CropType.RESIZE,
    )

    print(f"File: {file} Image shape: {image.shape}")
    predictions = model.predict(np.array([image]))
    # take the argmax of the predictions
    max_index = np.argmax(predictions)

    print(f"Prediction: {predictions} Max: {max_index}")

File: FAKE/0 (2).jpg Image shape: (224, 224, 3)


2025-01-07 12:56:55.517526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step
Prediction: [[2.8153814e-03 6.9535006e-04 9.9648929e-01]] Max: 2
File: FAKE/0 (3).jpg Image shape: (224, 224, 3)
1/1 [==============================] - 0s 141ms/step
Prediction: [[7.697749e-04 6.727223e-04 9.985575e-01]] Max: 2
File: FAKE/0 (4).jpg Image shape: (224, 224, 3)
1/1 [==============================] - 0s 129ms/step
Prediction: [[2.6467521e-04 4.1395988e-04 9.9932134e-01]] Max: 2
File: FAKE/0 (5).jpg Image shape: (224, 224, 3)
1/1 [==============================] - 0s 146ms/step
Prediction: [[1.5719245e-04 6.9890106e-05 9.9977297e-01]] Max: 2
File: FAKE/0 (6).jpg Image shape: (224, 224, 3)
1/1 [==============================] - 0s 129ms/step
Prediction: [[0.00128217 0.00166088 0.99705696]] Max: 2
File: FAKE/0 (7).jpg Image shape: (224, 224, 3)
1/1 [==============================] - 0s 129ms/step
Prediction: [[5.199854e-04 6.420963e-04 9.988379e-01]] Max: 2
File: FAKE/0 (8).jpg Image shape: (224, 224, 3)
1/1 [=================

In [6]:
# Upload model to huggingface

from huggingface_hub import push_to_hub_keras

push_to_hub_keras(
    model,
    repo_id=REPO_ID,
    tags=["0.1", "classification"],
)

In [7]:
from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras(REPO_ID, token=True, force_download=True)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
# extract prefix path from TEST_FILES_METADATA_CSV
prefx_path = Path(TEST_FILES_METADATA_CSV).parent
for file in test_files:
    file = file[0]
    image = preprocess_image(
        os.path.join(prefx_path, file),
        CropType.RESIZE,
    )

    print(f"File: {file} Image shape: {image.shape}")
    predictions = model(np.array([image]))
    # take the argmax of the predictions
    max_index = np.argmax(predictions)

    print(f"Prediction: {predictions} Max: {max_index}")

File: FAKE/0 (2).jpg Image shape: (224, 224, 3)
Prediction: [[2.8153814e-03 6.9535006e-04 9.9648929e-01]] Max: 2
File: FAKE/0 (3).jpg Image shape: (224, 224, 3)
Prediction: [[7.697749e-04 6.727223e-04 9.985575e-01]] Max: 2
File: FAKE/0 (4).jpg Image shape: (224, 224, 3)
Prediction: [[2.6467521e-04 4.1395988e-04 9.9932134e-01]] Max: 2
File: FAKE/0 (5).jpg Image shape: (224, 224, 3)
Prediction: [[1.5719245e-04 6.9890106e-05 9.9977297e-01]] Max: 2
File: FAKE/0 (6).jpg Image shape: (224, 224, 3)
Prediction: [[0.00128217 0.00166088 0.99705696]] Max: 2
File: FAKE/0 (7).jpg Image shape: (224, 224, 3)
Prediction: [[5.199854e-04 6.420963e-04 9.988379e-01]] Max: 2
File: FAKE/0 (8).jpg Image shape: (224, 224, 3)
Prediction: [[0.001379   0.00103081 0.9975902 ]] Max: 2
File: FAKE/0 (9).jpg Image shape: (224, 224, 3)
Prediction: [[0.00428889 0.00113533 0.99457574]] Max: 2
File: FAKE/0.jpg Image shape: (224, 224, 3)
Prediction: [[0.02382903 0.00507597 0.971095  ]] Max: 2
File: FAKE/1 (10).jpg Image s